In [ ]:
import re
from pathlib import Path

def clean_csv_record(line: str) -> str:
    """
    Очищает строку CSV:
    - Убирает лишние пробелы
    - Убирает пробелы вокруг запятых и знака равенства
    """
    line = line.rstrip('\r\n')
    line = re.sub(r"\s{2,}", " ", line)
    line = re.sub(r"\s*,\s*", ",", line)
    line = re.sub(r"\s*=\s*", "=", line)
    return line


def process_file(input_path: Path, output_path: Path) -> None:
    """
    Читает input_path, соединяет разбитые записи и записывает очищенный CSV в output_path.
    Каждая запись на отдельной строке, записи начинаются с числа.
    """
    new_record = re.compile(r"^\d+,")
    output_lines = []
    buffer = []

    with input_path.open('r', encoding='utf-8') as fin:
        for raw in fin:
            cleaned = clean_csv_record(raw)
            if new_record.match(cleaned):
                if buffer:
                    # добавляем накопленную запись
                    output_lines.append(''.join(buffer))
                buffer = [cleaned]
            else:
                buffer.append(cleaned)

    if buffer:
        output_lines.append(''.join(buffer))

    output_path.parent.mkdir(parents=True, exist_ok=True)
    with output_path.open('w', encoding='utf-8', newline='\n') as fout:
        fout.write("\n".join(output_lines))


def start_parsing():
    raw_dir = Path('./исходные данные')
    out_dir = Path('./после парсинга')
    pattern = 'MOCK_DATA*.csv'

    files = sorted(raw_dir.glob(pattern))
    if not files:
        print(f"Нет файлов по шаблону {pattern} в {raw_dir}")
        return

    for inp in files:
        out_name = f"parsed_mock{files.index(inp)+1}.csv"
        out_path = out_dir / out_name
        print(f"Обработка {inp} → {out_path}")
        process_file(inp, out_path)

    print("Готово!")

In [ ]:
import os
import glob
import psycopg2

DB_PARAMS = {
    "host":     os.getenv("DB_HOST", "localhost"),
    "port":     os.getenv("DB_PORT", "5432"),
    "dbname":   os.getenv("DB_NAME", "spark_db"),
    "user":     os.getenv("DB_USER", "spark_user"),
    "password": os.getenv("DB_PASSWORD", "spark_password"),
}

conn = psycopg2.connect(**DB_PARAMS)
cur = conn.cursor()

CSV_DIR = os.getenv("CSV_DIR", ".")
WORK_DIR = os.getenv("WORK_DIR", ".")

def load_file_contents(path):
    for enc in ("utf-8-sig", "utf-8", "cp1251", "latin-1"):
        try:
            with open(path, "r", encoding=enc) as f:
                return f.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Не удалось прочитать файл {path} в поддерживаемой кодировке")

def init_db():
    ddl_path = os.path.join(WORK_DIR, "init.sql")
    ddl_sql = load_file_contents(ddl_path)
    cur.execute(ddl_sql)
    conn.commit()
    print("DDL выполнен")

def load_data():
    pattern = os.path.join(CSV_DIR, "./после парсинга/parsed_mock*.csv")
    files = sorted(glob.glob(pattern))
    for path in files:
        print(f"Загрузка {path}")
        with open(path, "r", encoding="utf-8") as f:
            cur.copy_expert("COPY mock_data FROM STDIN WITH CSV HEADER", f)
    conn.commit()
    print("CSV загружены в mock_data")

def load():
    init_db()
    load_data()

